# Import Libraries

In [76]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns
import pickle
from datetime import timedelta, datetime
import datetime
import os
from dotenv import load_dotenv

# YouTube Connection

In [77]:
load_dotenv()

api_key = os.getenv('API_KEY')

youtube = build('youtube', 'v3', developerKey=api_key)

# Importing Data

In [78]:
df = pd.read_csv('cloud_function_data_youtube_video_data_2023-12-25 23_00_40.csv')
df.head()

,video_id,channel_id,published_at,title,description,tags,category_id,duration,caption,licensed_content,default_language,content_rating,view_count,like_count,favourite_count,comment_count,extraction_date
0,LciDiBeBCyY,UCteRPiisgIoHtMgqHegpWAQ,2023-12-15T15:00:16Z,How to Build Custom ChatGPTs to Learn FAST in ...,Get your Excel data visualization template (fr...,"['data science', 'data scientist', 'self-taugh...",28,PT12M,True,True,en-US,{},4240,225,0,18,2023-12-25 23:00:40
1,56i1uBshzmA,UCJublDh2UsiIKsAE1553miw,2023-12-15T13:00:08Z,My Favorite Stack Question! | Daily Temperatur...,"leetcode, coding interview question, data stru...","['leetcode', 'coding interview question', 'dat...",27,PT58S,False,True,en,{},28854,1161,0,19,2023-12-25 23:00:40
2,OPyoXx0yA0I,UCVhQ2NnY5Rskt6UjCUkJ_DA,2023-12-15T16:00:16Z,Requests vs Httpx vs Aiohttp | Which One to Pick?,"Exploring API communication in your app, consi...","['requests vs httpx', 'httpx', 'python request...",27,PT15M11S,True,True,en-US,{},20779,1316,0,97,2023-12-25 23:00:40
3,WjhKxXCwFZA,UCzL_0nIe8B4-7ShhVPfJkgw,2023-12-15T18:47:11Z,Data Analytics and Generative AI,Generative AI is a rapidly evolving field with...,"['data analytics', 'generative ai', 'large lan...",27,PT55M18S,True,False,en-US,{},266,6,0,1,2023-12-25 23:00:40
4,NI1Psgs1tyI,UCzL_0nIe8B4-7ShhVPfJkgw,2023-12-15T16:35:30Z,Enterprise LLM Applications - Not Just a Techn...,Generative Al and Large Language Models have t...,[],24,P0D,False,False,en-US,{},0,0,0,0,2023-12-25 23:00:40


# Day Function

In [79]:
def day_gen(data):

    # converting published at
    data['published_at_formatted'] = data['published_at'].str.replace('Z','')
    data['published_at_formatted'] = data.published_at_formatted.apply(datetime.datetime.fromisoformat)

    # converting extraction date
    data['extraction_date_formatted'] = pd.to_datetime(data['extraction_date'], format='%Y-%m-%d %H:%M:%S')

    # creating Day column
    data['Day'] =  ((data['extraction_date_formatted'] - data['published_at_formatted']).dt.days) + 1

    data.drop(columns=['published_at', 'extraction_date'], inplace=True)

    return data

# Cleaning Function

In [80]:
def clean(data):
    
    # Columns to String
    columns_to_str = ['video_id', 'channel_id', 'title', 'description']

    for column in columns_to_str:
        data[column] = data[column].astype('string')

    # Object to Int
    columns_to_int = ['view_count', 'like_count', 'comment_count']

    for column in columns_to_int:
        data[column] = data[column].astype('int64')

    # YouTube Categories
    def get_categories(youtube, wanted_categories):

        all_data = []
        
        request = youtube.videoCategories().list(part='snippet', id=','.join(wanted_categories))
        
        response = request.execute()
        
        for i in range(len(response['items'])):
            data = dict(category_id = response['items'][i]['id'],
                    category= response['items'][i]['snippet']['title'])
            all_data.append(data)

        return all_data
    
    data['category_id'] = data['category_id'].astype('str')
    category_ids = data['category_id'].unique()
    category_dict = get_categories(youtube, category_ids)
    category_df = pd.DataFrame(category_dict)
    data = pd.merge(data, category_df, on='category_id', how='left')

    # Extracting Duration
    data['duration'] = data['duration'].str.replace('PT', '')
    def format_duration(time_string):

        # store hours, minutes, seconds as integers
        H = 0
        M = 0
        S = 0

        # check if vid time contains hours, minutes and/or seconds
        if 'H' in time_string:
            H += int(time_string.split('H')[0])
        if 'M' in time_string:
            M += int(time_string.split('M')[0].split('H')[-1])
        if 'S' in time_string:
            S += int(time_string.split('S')[0].split('M')[-1].split('H')[-1])
        
        formatted_time = timedelta(hours=H, minutes=M, seconds=S)

        return formatted_time

    data['duration_formatted'] = data['duration'].apply(format_duration)

    # Object to Bool
    columns_to_bool = ['caption', 'licensed_content']

    for column in columns_to_bool:
        data[column] = data[column].astype('bool')

    # Creating No. of Tags Column
    data['no_of_tags'] = data['tags'].apply(lambda x: len(set(x)))

    # Creating Title Length Column
    data['title_length'] = data['title'].apply(len)

    # Creating Description Length Column
    data['description_length'] = data['description'].apply(len)

    # Dropping Columns
    columns_to_drop = ['category_id', 'duration', 'content_rating']
    data.drop(columns=columns_to_drop, inplace=True)

    return data

# Testing Cleaning function on OG df

In [81]:
clean_df = clean(df)

In [82]:
clean_df = day_gen(clean_df)

In [83]:
clean_df.head()

,video_id,channel_id,title,description,tags,caption,licensed_content,default_language,view_count,like_count,favourite_count,comment_count,category,duration_formatted,no_of_tags,title_length,description_length,published_at_formatted,extraction_date_formatted,Day
0,LciDiBeBCyY,UCteRPiisgIoHtMgqHegpWAQ,How to Build Custom ChatGPTs to Learn FAST in ...,Get your Excel data visualization template (fr...,"['data science', 'data scientist', 'self-taugh...",True,True,en-US,4240,225,0,18,Science & Technology,0 days 00:12:00,36,52,4537,2023-12-15 15:00:16,2023-12-25 23:00:40,11
1,56i1uBshzmA,UCJublDh2UsiIKsAE1553miw,My Favorite Stack Question! | Daily Temperatur...,"leetcode, coding interview question, data stru...","['leetcode', 'coding interview question', 'dat...",False,True,en,28854,1161,0,19,Education,0 days 00:00:58,26,63,91,2023-12-15 13:00:08,2023-12-25 23:00:40,11
2,OPyoXx0yA0I,UCVhQ2NnY5Rskt6UjCUkJ_DA,Requests vs Httpx vs Aiohttp | Which One to Pick?,"Exploring API communication in your app, consi...","['requests vs httpx', 'httpx', 'python request...",True,True,en-US,20779,1316,0,97,Education,0 days 00:15:11,24,49,2023,2023-12-15 16:00:16,2023-12-25 23:00:40,11
3,WjhKxXCwFZA,UCzL_0nIe8B4-7ShhVPfJkgw,Data Analytics and Generative AI,Generative AI is a rapidly evolving field with...,"['data analytics', 'generative ai', 'large lan...",True,False,en-US,266,6,0,1,Education,0 days 00:55:18,21,32,1090,2023-12-15 18:47:11,2023-12-25 23:00:40,11
4,NI1Psgs1tyI,UCzL_0nIe8B4-7ShhVPfJkgw,Enterprise LLM Applications - Not Just a Techn...,Generative Al and Large Language Models have t...,[],False,False,en-US,0,0,0,0,Entertainment,0 days 00:00:00,2,64,1150,2023-12-15 16:35:30,2023-12-25 23:00:40,11


# Concat and Apply Days

In [84]:
def merge_clean_function(running_df, new_df):

    # cloud needs storage of names of dfs so it knows which one to add

    # cleaning the data

    clean_new_df = clean(new_df)

    # generate day column

    day_clean_new_df = day_gen(clean_new_df)
    
    # concat with running df

    running_df = pd.concat([running_df, day_clean_new_df])
    
    running_df.reset_index(drop=True, inplace=True)

    return running_df

# Testing with another days data

In [85]:
df_2 = pd.read_csv('cloud_function_data_youtube_video_data_2023-12-26 23_00_41.csv')
df_2.head()

,video_id,channel_id,published_at,title,description,tags,category_id,duration,caption,licensed_content,default_language,content_rating,view_count,like_count,favourite_count,comment_count,extraction_date
0,LciDiBeBCyY,UCteRPiisgIoHtMgqHegpWAQ,2023-12-15T15:00:16Z,How to Build Custom ChatGPTs to Learn FAST in ...,Get your Excel data visualization template (fr...,"['data science', 'data scientist', 'self-taugh...",28,PT12M,True,True,en-US,{},4367,232,0,18,2023-12-26 23:00:41
1,56i1uBshzmA,UCJublDh2UsiIKsAE1553miw,2023-12-15T13:00:08Z,My Favorite Stack Question! | Daily Temperatur...,"leetcode, coding interview question, data stru...","['leetcode', 'coding interview question', 'dat...",27,PT58S,False,True,en,{},29122,1170,0,20,2023-12-26 23:00:41
2,OPyoXx0yA0I,UCVhQ2NnY5Rskt6UjCUkJ_DA,2023-12-15T16:00:16Z,Requests vs Httpx vs Aiohttp | Which One to Pick?,"Exploring API communication in your app, consi...","['requests vs httpx', 'httpx', 'python request...",27,PT15M11S,True,True,en-US,{},21151,1338,0,97,2023-12-26 23:00:41
3,WjhKxXCwFZA,UCzL_0nIe8B4-7ShhVPfJkgw,2023-12-15T18:47:11Z,Data Analytics and Generative AI,Generative AI is a rapidly evolving field with...,"['data analytics', 'generative ai', 'large lan...",27,PT55M18S,True,False,en-US,{},270,6,0,1,2023-12-26 23:00:41
4,NI1Psgs1tyI,UCzL_0nIe8B4-7ShhVPfJkgw,2023-12-15T16:35:30Z,Enterprise LLM Applications - Not Just a Techn...,Generative Al and Large Language Models have t...,[],24,P0D,False,False,en-US,{},0,0,0,0,2023-12-26 23:00:41


In [86]:
test_running_df = merge_clean_function(clean_df, df_2)

In [87]:
clean_df.shape

(55, 20)

In [88]:
df_2.shape

(59, 17)

In [89]:
test_running_df.shape

(114, 20)

In [90]:
test_running_df.head()

,video_id,channel_id,title,description,tags,caption,licensed_content,default_language,view_count,like_count,favourite_count,comment_count,category,duration_formatted,no_of_tags,title_length,description_length,published_at_formatted,extraction_date_formatted,Day
0,LciDiBeBCyY,UCteRPiisgIoHtMgqHegpWAQ,How to Build Custom ChatGPTs to Learn FAST in ...,Get your Excel data visualization template (fr...,"['data science', 'data scientist', 'self-taugh...",True,True,en-US,4240,225,0,18,Science & Technology,0 days 00:12:00,36,52,4537,2023-12-15 15:00:16,2023-12-25 23:00:40,11
1,56i1uBshzmA,UCJublDh2UsiIKsAE1553miw,My Favorite Stack Question! | Daily Temperatur...,"leetcode, coding interview question, data stru...","['leetcode', 'coding interview question', 'dat...",False,True,en,28854,1161,0,19,Education,0 days 00:00:58,26,63,91,2023-12-15 13:00:08,2023-12-25 23:00:40,11
2,OPyoXx0yA0I,UCVhQ2NnY5Rskt6UjCUkJ_DA,Requests vs Httpx vs Aiohttp | Which One to Pick?,"Exploring API communication in your app, consi...","['requests vs httpx', 'httpx', 'python request...",True,True,en-US,20779,1316,0,97,Education,0 days 00:15:11,24,49,2023,2023-12-15 16:00:16,2023-12-25 23:00:40,11
3,WjhKxXCwFZA,UCzL_0nIe8B4-7ShhVPfJkgw,Data Analytics and Generative AI,Generative AI is a rapidly evolving field with...,"['data analytics', 'generative ai', 'large lan...",True,False,en-US,266,6,0,1,Education,0 days 00:55:18,21,32,1090,2023-12-15 18:47:11,2023-12-25 23:00:40,11
4,NI1Psgs1tyI,UCzL_0nIe8B4-7ShhVPfJkgw,Enterprise LLM Applications - Not Just a Techn...,Generative Al and Large Language Models have t...,[],False,False,en-US,0,0,0,0,Entertainment,0 days 00:00:00,2,64,1150,2023-12-15 16:35:30,2023-12-25 23:00:40,11


# Adding another Day

In [91]:
df_3 = pd.read_csv('cloud_function_data_youtube_video_data_2023-12-27 23_00_43.csv')
df_3.head()

,video_id,channel_id,published_at,title,description,tags,category_id,duration,caption,licensed_content,default_language,content_rating,view_count,like_count,favourite_count,comment_count,extraction_date
0,LciDiBeBCyY,UCteRPiisgIoHtMgqHegpWAQ,2023-12-15T15:00:16Z,How to Build Custom ChatGPTs to Learn FAST in ...,Get your Excel data visualization template (fr...,"['data science', 'data scientist', 'self-taugh...",28,PT12M,True,True,en-US,{},4457,233,0,18,2023-12-27 23:00:42
1,56i1uBshzmA,UCJublDh2UsiIKsAE1553miw,2023-12-15T13:00:08Z,My Favorite Stack Question! | Daily Temperatur...,"leetcode, coding interview question, data stru...","['leetcode', 'coding interview question', 'dat...",27,PT58S,False,True,en,{},29266,1179,0,20,2023-12-27 23:00:42
2,OPyoXx0yA0I,UCVhQ2NnY5Rskt6UjCUkJ_DA,2023-12-15T16:00:16Z,Requests vs Httpx vs Aiohttp | Which One to Pick?,"Exploring API communication in your app, consi...","['requests vs httpx', 'httpx', 'python request...",27,PT15M11S,True,True,en-US,{},21650,1357,0,97,2023-12-27 23:00:42
3,WjhKxXCwFZA,UCzL_0nIe8B4-7ShhVPfJkgw,2023-12-15T18:47:11Z,Data Analytics and Generative AI,Generative AI is a rapidly evolving field with...,"['data analytics', 'generative ai', 'large lan...",27,PT55M18S,True,False,en-US,{},277,6,0,2,2023-12-27 23:00:42
4,NI1Psgs1tyI,UCzL_0nIe8B4-7ShhVPfJkgw,2023-12-15T16:35:30Z,Enterprise LLM Applications - Not Just a Techn...,Generative Al and Large Language Models have t...,[],24,P0D,False,False,en-US,{},0,0,0,0,2023-12-27 23:00:42


In [92]:
test_2_running_df = merge_clean_function(test_running_df, df_3)

In [93]:
test_2_running_df.head()

,video_id,channel_id,title,description,tags,caption,licensed_content,default_language,view_count,like_count,favourite_count,comment_count,category,duration_formatted,no_of_tags,title_length,description_length,published_at_formatted,extraction_date_formatted,Day
0,LciDiBeBCyY,UCteRPiisgIoHtMgqHegpWAQ,How to Build Custom ChatGPTs to Learn FAST in ...,Get your Excel data visualization template (fr...,"['data science', 'data scientist', 'self-taugh...",True,True,en-US,4240,225,0,18,Science & Technology,0 days 00:12:00,36,52,4537,2023-12-15 15:00:16,2023-12-25 23:00:40,11
1,56i1uBshzmA,UCJublDh2UsiIKsAE1553miw,My Favorite Stack Question! | Daily Temperatur...,"leetcode, coding interview question, data stru...","['leetcode', 'coding interview question', 'dat...",False,True,en,28854,1161,0,19,Education,0 days 00:00:58,26,63,91,2023-12-15 13:00:08,2023-12-25 23:00:40,11
2,OPyoXx0yA0I,UCVhQ2NnY5Rskt6UjCUkJ_DA,Requests vs Httpx vs Aiohttp | Which One to Pick?,"Exploring API communication in your app, consi...","['requests vs httpx', 'httpx', 'python request...",True,True,en-US,20779,1316,0,97,Education,0 days 00:15:11,24,49,2023,2023-12-15 16:00:16,2023-12-25 23:00:40,11
3,WjhKxXCwFZA,UCzL_0nIe8B4-7ShhVPfJkgw,Data Analytics and Generative AI,Generative AI is a rapidly evolving field with...,"['data analytics', 'generative ai', 'large lan...",True,False,en-US,266,6,0,1,Education,0 days 00:55:18,21,32,1090,2023-12-15 18:47:11,2023-12-25 23:00:40,11
4,NI1Psgs1tyI,UCzL_0nIe8B4-7ShhVPfJkgw,Enterprise LLM Applications - Not Just a Techn...,Generative Al and Large Language Models have t...,[],False,False,en-US,0,0,0,0,Entertainment,0 days 00:00:00,2,64,1150,2023-12-15 16:35:30,2023-12-25 23:00:40,11


# Developing function for requesting database

### Testing Flip

In [149]:
df_a = pd.read_csv('cloud_function_data_youtube_video_data_2023-12-15 23_01_00 (2).csv')
df_b = pd.read_csv('cloud_function_data_youtube_video_data_2023-12-16 23_00_40 (1).csv')
df_c = pd.read_csv('cloud_function_data_youtube_video_data_2023-12-17 23_00_40 (1).csv')
df_d = pd.read_csv('cloud_function_data_youtube_video_data_2023-12-18 23_00_52 (1).csv')
df_e = pd.read_csv('cloud_function_data_youtube_video_data_2023-12-19 23_00_41.csv')

In [150]:
clean_a = clean(df_a)
final_a = day_gen(clean_a)

In [151]:
test_flip_df = merge_clean_function(final_a, df_b)
test_flip_df = merge_clean_function(test_flip_df, df_c)
test_flip_df = merge_clean_function(test_flip_df, df_d)
test_flip_df = merge_clean_function(test_flip_df, df_e)

In [147]:
def get_channel_stats(youtube_key, channel_ids):

    all_data = []

    request = youtube_key.channels().list(part='snippet,contentDetails,statistics', id=','.join(channel_ids))

    response = request.execute()

    for i in range(len(response['items'])):
        data = dict(channel_name = response['items'][i]['snippet']['title'],
                channel_id = response['items'][i]['id'],
                subscribers = response['items'][i]['statistics']['subscriberCount'],
                total_views = response['items'][i]['statistics']['viewCount'],
                total_videos = response['items'][i]['statistics']['videoCount'],
                playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    channel_stats_df = pd.DataFrame(all_data)

    return channel_stats_df

In [148]:
def database_creator(data, days):

    # Filter by requested days

    filter_list = list(data[data['Day'] == int(days)]['video_id'])
    filtered_df = data[data['video_id'].isin(filter_list)]
    filtered_df = filtered_df[filtered_df['Day'] <= int(days)]

    # Flip
    filtered_df['Day'] = 'Day' + filtered_df['Day'].astype(str)
    df_pivoted = filtered_df.pivot_table(index='video_id', columns='Day', values=['view_count', 'like_count', 'comment_count'], aggfunc='first', fill_value=0).reset_index()

    columns_to_keep = ['video_id', 'channel_id', 'title', 'description', 'tags', 'caption',
        'licensed_content', 'default_language','category', 'duration_formatted',
        'no_of_tags', 'title_length', 'description_length',
        'published_at_formatted', 'extraction_date_formatted']
    df_unique_ids = filtered_df[columns_to_keep].copy()

    df_unique_ids.drop_duplicates(subset='video_id', keep='first', inplace=True)

    df_unique_ids.reset_index(drop=True, inplace=True)

    # Merge with the original DataFrame to include other information
    result_df = df_pivoted.merge(df_unique_ids, on='video_id', how='left')
    result_df.drop(columns=[('video_id', '')], inplace=True)


    # get list of channel ids of the videos
    channel_ids_list = list(result_df['channel_id'])
    
    # get channel stats dataframe
    channel_df = get_channel_stats(youtube, channel_ids_list)

    # merge final dataframe
    final_df = pd.merge(result_df, channel_df, on='channel_id', how='left')

    return final_df


In [152]:
test_database = database_creator(test_flip_df, 3)

C:\Users\mms\AppData\Local\Temp\ipykernel_16452\2951152658.py:25: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  result_df = df_pivoted.merge(df_unique_ids, on='video_id', how='left')


In [154]:
test_database.shape

(12, 29)